In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('credit_risk_small.csv')

In [3]:
df.head()

,age,income,loan_amount,credit_score,debt_to_income,employment_status,home_ownership,purpose,default
0,56,41905.0,11795.0,632,0.07,employed,rent,medical,0
1,69,21258.0,9685.0,695,0.05,employed,mortgage,education,0
2,46,69177.0,1133.0,606,0.56,employed,rent,debt_consolidation,1
3,32,37700.0,15558.0,638,0.49,self-employed,rent,education,1
4,60,26395.0,17482.0,611,0.39,employed,mortgage,car,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                200 non-null    int64  
 1   income             200 non-null    float64
 2   loan_amount        200 non-null    float64
 3   credit_score       200 non-null    int64  
 4   debt_to_income     200 non-null    float64
 5   employment_status  200 non-null    object 
 6   home_ownership     200 non-null    object 
 7   purpose            200 non-null    object 
 8   default            200 non-null    int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 14.2+ KB


In [9]:
df['employment_status'].value_counts()

employment_status
employed         119
self-employed     33
unemployed        33
student           15
Name: count, dtype: int64

In [10]:
df['home_ownership'].value_counts()

home_ownership
rent        103
mortgage     59
own          38
Name: count, dtype: int64

In [11]:
df['purpose'].value_counts()

purpose
debt_consolidation    78
medical               43
education             40
car                   39
Name: count, dtype: int64

In [12]:
X = df.drop(columns=['default'])
y = df['default']

In [14]:
X = pd.get_dummies(df,columns=['employment_status','home_ownership','purpose'], drop_first=True, dtype=int)

In [15]:
X.head()

,age,income,loan_amount,credit_score,debt_to_income,default,employment_status_self-employed,employment_status_student,employment_status_unemployed,home_ownership_own,home_ownership_rent,purpose_debt_consolidation,purpose_education,purpose_medical
0,56,41905.0,11795.0,632,0.07,0,0,0,0,0,1,0,0,1
1,69,21258.0,9685.0,695,0.05,0,0,0,0,0,0,0,1,0
2,46,69177.0,1133.0,606,0.56,1,0,0,0,0,1,1,0,0
3,32,37700.0,15558.0,638,0.49,1,1,0,0,0,1,0,1,0
4,60,26395.0,17482.0,611,0.39,1,0,0,0,0,0,0,0,0


In [16]:
X.columns

Index(['age', 'income', 'loan_amount', 'credit_score', 'debt_to_income',
       'default', 'employment_status_self-employed',
       'employment_status_student', 'employment_status_unemployed',
       'home_ownership_own', 'home_ownership_rent',
       'purpose_debt_consolidation', 'purpose_education', 'purpose_medical'],
      dtype='object')

In [17]:
y.value_counts()

default
0    158
1     42
Name: count, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [19]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators = 100
)

model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

In [20]:
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)

In [22]:
df.shape

(200, 9)

In [23]:
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))

1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        30
           1       1.00      1.00      1.00        10

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

[[30  0]
 [ 0 10]]
